# Upload Synonym to neo4j

In [1]:
from pyspark.sql import SparkSession
from data_class.Jesmine_Network_of_Synonym import Network_of_Synonym
from data_class.Neo4j import Neo4jQuery
from uuid import uuid4

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("CSV to Neo4j") \
    .getOrCreate()

# Initialize Neo4j connection
URI = "neo4j+s://2257c2b1.databases.neo4j.io"  ###Remember to replace
neo4j_user = "neo4j"
neo4j_password = "9GMnG4B136sU9Onf8VugQuULe2UMZ7vPjeuD-1nCqbQ" ###Remember to replace

# Create a Neo4jConnection instance
neo4j_conn = Neo4jQuery(uri=URI, user=neo4j_user, password=neo4j_password)

24/12/21 18:01:04 WARN Utils: Your hostname, MSI. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/21 18:01:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 18:01:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read CSV file from HDFS
df = spark.read.csv(
    "sinonim.csv", 
    header=True,         
    inferSchema=True,   
    multiLine=True,     
    sep=","         
)

# Collect all rows into a list of dictionaries
nodes = df.collect()

create_synonym_query = """
MERGE (word:Word {name: $word})
WITH word
UNWIND $synonyms AS synonym
MERGE (synonymNode:Word {name: synonym})
MERGE (word)-[:SYNONYM_OF]->(synonymNode)
"""

for row in nodes:
    tema = row["kata"]
    sinonim_list = row["sinonim"].split(", ")  

    neo4j_conn.execute_query(
        query=create_synonym_query,
        parameters={"word": tema, "synonyms": sinonim_list}
    )

print("Data successfully uploaded to Neo4j!")

In [4]:
neo4j_conn.close()
spark.stop()